In [1]:
import numpy as np
import pandas as pd
import keras

In [2]:
import matplotlib.pyplot as plt
%matplotlib tk

In [3]:
import os

In [4]:
modelMSE = keras.saving.load_model(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\modeli\Difference predictions\X - GRU")

In [5]:
path = r"C:\Users\mata2\Desktop\master\podaci\novi_deo\x-0k"

In [6]:
dir = os.listdir(path)

Ucitavanje fajlova

In [7]:
file_names = []
for file in dir:
    file_names.append(file)

In [8]:
test_file_names = file_names

Ucitavanje zadate trajektorije

In [9]:
x_command = np.asarray(pd.read_csv(r"C:\Users\mata2\Desktop\master\podaci\novi_deo\xCmd_novi_deo.txt",header=None).astype('int'))

Konkatenacija fajlova koji se posle pravilno rasporedjuju na adekvatne train/test skupove

In [10]:
test_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:len(x_command)].astype('int')
                              for file in test_file_names], axis=0)

Prethodno konkatenizovani podaci se adekvatno dele kako bi se izbeglo fiktivno stanje (stanje pri kraju rada i pocetku novog ako su konkatenizovani podaci)

In [11]:
for j in range(len(test_file_names)):
    exec(f"razlika_test_{j} = []")
    for i in range(len(x_command)):
        exec(f"razlika_test_{j}.append(x_command[i] - test_[j*len(x_command)+i])")
    exec(f'razlika_test_{j} = np.asarray(razlika_test_{j})')    

Funkcija koja deli podatke na "sliding window"

In [13]:
def create_timeseries_split(data, time_steps):

  # Get the number of samples in the data
  num_samples = len(data) - time_steps

  # Create empty arrays to store features and target values
  x_train = np.zeros((num_samples, time_steps, 1))
  y_train = np.zeros((num_samples, 1))

  # Loop through the data and create features and target values
  for i in range(num_samples):
    # Extract a slice of data for the current feature
    x_train[i] = data[i:i+time_steps, :]

    # The target value is the next value after the feature
    y_train[i] = data[i+time_steps, 0]

  return x_train, y_train

Buffer

In [12]:
time_steps = 8

Deljenje podataka na train i test skup

In [14]:
for i in range(len(test_file_names)):
    exec(f'test_X_{i}, test_y_{i} = create_timeseries_split(razlika_test_{i}, time_steps)')

test_X = np.asarray(test_X_0)
for i in range(1,len(test_file_names)):
    exec(f'test_X = np.append(test_X, test_X_{i}, axis=0)')

test_y = np.asarray(test_y_0)
for i in range(1,len(test_file_names)):
    exec(f'test_y = np.append(test_y, test_y_{i}, axis=0)')

In [15]:
input_shape = (time_steps,1)

In [16]:
test_predict_MSE = modelMSE.predict(test_X)

16872/16872 [==============================] - 26s 2ms/step


Ucitavamo standardne devijacije i srednje vrednosti

In [18]:
std_dev = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k - x novi podaci\0k - x novi podaciX_std_dev.txt", 'r') as lines:
    for line in lines:
        std_dev.append(float(line.strip('\n')))

In [17]:
sr_vrednost = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k - x novi podaci\0k - x novi podaciX_sr_vrednost.txt", 'r') as lines:
    for line in lines:
        sr_vrednost.append(float(line.strip('\n')))

In [19]:
sig_len = int(len(test_)/len(test_file_names))

In [21]:
mean_1std = 0
for j in range(len(test_file_names)):
    for i in range(int(len(test_predict_MSE)/len(test_file_names))):
        if sr_vrednost[i+time_steps] - std_dev[i+time_steps] <= test_predict_MSE[int(len(test_predict_MSE)/len(test_file_names))*j + i] <= sr_vrednost[i+time_steps] + std_dev[i+time_steps]:
            mean_1std += 1
        #else:
            #print(sr_vrednost[i+8], std_dev[i+8], test_predict_MSE[60992*j + i], 'index:',j*60992 + i)
mean_1std/len(test_predict_MSE)

0.6829591761132103

Mean 3 sigma

In [22]:
i = time_steps
mean_3std = 0
for j in range(len(test_predict_MSE)):
    if i%len(x_command) == 0:
        i = time_steps
    if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
            mean_3std += 1
    i += 1
mean_3std/len(test_predict_MSE)

0.9961565533081426

In [20]:
min_kord = [float('inf') for x in range(sig_len)]
max_kord = [float('-inf') for x in range(sig_len)]

for i in range(sig_len):    
    for j in range(len(test_file_names)):
        exec(f'current_file = razlika_test_{j}')
        if current_file[i][0] < min_kord[i]:
            min_kord[i] = current_file[i][0]
        if current_file[i][0] > max_kord[i]:
            max_kord[i] = current_file[i][0]

In [62]:
i = time_steps
uspeli_minmax = 0
for j in range(len(test_predict_MSE)):
    if i%sig_len == 0:
        i = time_steps
    if min_kord[i] <= (test_predict_MSE[j]) <= max_kord[i]:
        uspeli_minmax += 1
    i += 1
uspeli_minmax/len(test_predict_MSE)

0.8896884492850263

# NA TEST SKUPU - plotovi trajektorija

test_ = realizovana trajektorija

test_predict_MSE = predvidjena RAZLIKA koordinata od x_command

razlika_test_{} = realizovana RAZLIKA od test podataka od x_command - CELA TRAJEKTORIJA

test_y_{} = REALIZOVANA y koordinata od razlika_test_{}

X_COMMAND - TEST_ = ODSTUPANJE/RAZLIKA_TEST_

X_COMMAND[TIME_STEPS] + test_predict_MSE[0] = PRVU KOORDINATU TRAJEKTORIJE PREDVIDJENE VREDNOSTI

# tri sigma plotovi

In [21]:
tri_sigma_lower = []
tri_sigma_upper = []

for i in range(len(x_command)):
    tri_sigma_lower.append(sr_vrednost[i] - 3*std_dev[i])
    tri_sigma_upper.append(sr_vrednost[i] + 3*std_dev[i])

In [22]:
for j in range(len(test_file_names)):
    exec(f'trajektorija_odstupanja_{j} = []')
    for i in range(sig_len - time_steps):
        exec(f'trajektorija_odstupanja_{j}.append(test_predict_MSE[j*(sig_len-time_steps) + i])')

In [23]:
trajektorija = 1
i = time_steps
mean_3std = 0
for j in range(len(test_predict_MSE)):
    if i%len(x_command) == 0:
        print("trajektorija ",trajektorija, " sadrzi ", mean_3std , " tacaka", ", procenat tri sigma iznosi: ",round(mean_3std/36000*100,2),"%")
        exec(f"trajektorija_{trajektorija} = mean_3std")
        trajektorija += 1
        mean_3std = 0
        i = time_steps
    if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
            mean_3std += 1
    i += 1
trajektorija_15 = mean_3std
print("trajektorija ",trajektorija, " sadrzi ", mean_3std , " tacaka", ", procenat tri sigma iznosi: ",round(mean_3std/36000*100,2),"%")

trajektorija  1  sadrzi  35930  tacaka , procenat tri sigma iznosi:  99.81 %
trajektorija  2  sadrzi  35750  tacaka , procenat tri sigma iznosi:  99.31 %
trajektorija  3  sadrzi  35895  tacaka , procenat tri sigma iznosi:  99.71 %
trajektorija  4  sadrzi  35814  tacaka , procenat tri sigma iznosi:  99.48 %
trajektorija  5  sadrzi  35839  tacaka , procenat tri sigma iznosi:  99.55 %
trajektorija  6  sadrzi  35860  tacaka , procenat tri sigma iznosi:  99.61 %
trajektorija  7  sadrzi  35930  tacaka , procenat tri sigma iznosi:  99.81 %
trajektorija  8  sadrzi  35834  tacaka , procenat tri sigma iznosi:  99.54 %
trajektorija  9  sadrzi  35831  tacaka , procenat tri sigma iznosi:  99.53 %
trajektorija  10  sadrzi  35875  tacaka , procenat tri sigma iznosi:  99.65 %
trajektorija  11  sadrzi  35885  tacaka , procenat tri sigma iznosi:  99.68 %
trajektorija  12  sadrzi  35882  tacaka , procenat tri sigma iznosi:  99.67 %
trajektorija  13  sadrzi  35796  tacaka , procenat tri sigma iznosi:  99.

In [25]:
plt.figure(figsize=(11,7))
plt.plot(tri_sigma_lower[time_steps:], color= 'b')
plt.plot(tri_sigma_upper[time_steps:], color = 'r')
plt.plot(trajektorija_odstupanja_0, color = 'g')
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.legend(['три сигма доња граница',"три сигма горња граница", "одступање трајекторије 1"],loc = 'best')
plt.show()

In [26]:
plt.figure(figsize=(11,7))
for i in range(len(trajektorija_odstupanja_0)):
    if tri_sigma_lower[time_steps + i] <= trajektorija_odstupanja_0[i] <= tri_sigma_upper[time_steps + i]:
        plt.plot(i, trajektorija_odstupanja_0[i], color='black', marker = 'o', markersize = 2)
    else:
        plt.plot(i, trajektorija_odstupanja_0[i], color='red', marker = 'o', markersize = 2)
plt.title('Количина тачака која упада у три сигма појас износи: {}%'.format(round((trajektorija_1/36000)*100,2)) , fontsize=18)
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.show()

In [27]:
plt.figure(figsize=(11,7))
for i in range(len(trajektorija_odstupanja_1)):
    if tri_sigma_lower[time_steps + i] <= trajektorija_odstupanja_1[i] <= tri_sigma_upper[time_steps + i]:
        plt.plot(i, trajektorija_odstupanja_1[i], color='black', marker = 'o', markersize = 2)
    else:
        plt.plot(i, trajektorija_odstupanja_1[i], color='red', marker = 'o', markersize = 2)
plt.title('Количина тачака која упада у три сигма појас износи: {}%'.format(round((trajektorija_2/36000)*100,2)) , fontsize=18)
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.show()

In [40]:
plt.figure(figsize=(11,7))
for i in range(len(trajektorija_odstupanja_2)):
    if tri_sigma_lower[time_steps + i] <= trajektorija_odstupanja_2[i] <= tri_sigma_upper[time_steps + i]:
        plt.plot(i, trajektorija_odstupanja_2[i], color='black', marker = 'o', markersize = 2)
    else:
        plt.plot(i, trajektorija_odstupanja_2[i], color='red', marker = 'o', markersize = 2)


# min maks plotovi

In [28]:
min_kord = [float('inf') for x in range(sig_len)]
max_kord = [float('-inf') for x in range(sig_len)]

for i in range(sig_len):    
    for j in range(len(test_file_names)):
        exec(f'current_file = razlika_test_{j}')
        if current_file[i][0] < min_kord[i]:
            min_kord[i] = current_file[i][0]
        if current_file[i][0] > max_kord[i]:
            max_kord[i] = current_file[i][0]

In [29]:
min_trajektorija = []
max_trajektorija = []
for i in range(sig_len):
    min_trajektorija.append(x_command[i] + min_kord[i])
    max_trajektorija.append(x_command[i] + max_kord[i])

In [30]:
for j in range(len(test_file_names)):
    exec(f'predvidjena_trajektorija_{j} = []')
    for i in range(len(x_command) - time_steps):
        exec(f'predvidjena_trajektorija_{j}.append(x_command[i + time_steps] + test_predict_MSE[j*(len(x_command)-time_steps) + i])')

In [32]:
trajektorija_koord = 1
i = time_steps
mean_3std_koord = 0
for j in range(len(test_predict_MSE)):
    if i%len(x_command) == 0:
        print("trajektorija ",trajektorija_koord, " sadrzi ", mean_3std_koord , " tacaka", ", procenat tri sigma iznosi: ",round(mean_3std_koord/36000*100,2), "%")
        exec(f"trajektorija_koord_{trajektorija_koord} = mean_3std_koord")
        trajektorija_koord += 1
        mean_3std_koord = 0
        i = time_steps
    if min_kord[i] <= (test_predict_MSE[j]) <= max_kord[i]:
            mean_3std_koord += 1
    i += 1
trajektorija_koord_15 = mean_3std_koord
print("trajektorija ",trajektorija_koord, " sadrzi ", mean_3std_koord , " tacaka", ", procenat tri sigma iznosi: ",round(mean_3std_koord/36000*100,2),"%")

trajektorija  1  sadrzi  31393  tacaka , procenat tri sigma iznosi:  87.2 %
trajektorija  2  sadrzi  28751  tacaka , procenat tri sigma iznosi:  79.86 %
trajektorija  3  sadrzi  31525  tacaka , procenat tri sigma iznosi:  87.57 %
trajektorija  4  sadrzi  32540  tacaka , procenat tri sigma iznosi:  90.39 %
trajektorija  5  sadrzi  31514  tacaka , procenat tri sigma iznosi:  87.54 %
trajektorija  6  sadrzi  33978  tacaka , procenat tri sigma iznosi:  94.38 %
trajektorija  7  sadrzi  31393  tacaka , procenat tri sigma iznosi:  87.2 %
trajektorija  8  sadrzi  33434  tacaka , procenat tri sigma iznosi:  92.87 %
trajektorija  9  sadrzi  32991  tacaka , procenat tri sigma iznosi:  91.64 %
trajektorija  10  sadrzi  33093  tacaka , procenat tri sigma iznosi:  91.92 %
trajektorija  11  sadrzi  31820  tacaka , procenat tri sigma iznosi:  88.39 %
trajektorija  12  sadrzi  33190  tacaka , procenat tri sigma iznosi:  92.19 %
trajektorija  13  sadrzi  30632  tacaka , procenat tri sigma iznosi:  85.09

In [33]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_0)):
    if i%36000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_0[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_0[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_0[j], color='red',marker='o',markersize=2)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање X осе машине (μm)', fontsize=12)
plt.title('Количина тачака која упада у мин макс појас износи: {}%'.format(round((trajektorija_koord_1/36000)*100,2)) , fontsize=18)
plt.show()

In [34]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_1)):
    if i%36000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_1[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_1[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_1[j], color='red',marker='o',markersize=2)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање X осе машине (μm)', fontsize=12)
plt.title('Количина тачака која упада у мин макс појас износи: {}%'.format(round((trajektorija_koord_2/36000)*100,2)) , fontsize=18)
plt.show()